In [1]:
#@title ⚙️ Ejecutar esta celda para instalar las librería.
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""

!pip install git+https://github.com/openai/whisper.git
!pip install jiwer

from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-9bvfoe__
  Running command git clone -q https://github.com/openai/whisper.git /tmp/pip-req-build-9bvfoe__
     |████████████████████████████████| 5.8 MB 20.7 MB/s 
     |████████████████████████████████| 182 kB 68.4 MB/s 
     |████████████████████████████████| 7.6 MB 49.1 MB/s 
  Created wheel for whisper: filename=whisper-1.0-py3-none-any.whl size=1175324 sha256=5c8b868f78ec3b0a0de27e0d39b335a833facf313cb3622fddeef477f89e9678
  Stored in directory: /tmp/pip-ephem-wheel-cache-poky6y5r/wheels/a7/70/18/b7693c07b1d18b3dafb328f5d0496aa0d41a9c09ef332fd8e6
Successfully built whisper
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 31.6 MB/s 
     |████████████████████████████████| 2.2 MB 66.7 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [4]:
!whisper "/content/TrueSurvivor.mp3" --task transcribe --model medium --verbose False --output_dir audio_transcription

tcmalloc: large alloc 1528012800 bytes == 0xba04000 @  0x7fc2cd2331e7 0x4d3280 0x5de162 0x60fd9f 0x5a9e9b 0x46eb22 0x616b6b 0x4f7ada 0x49ca7c 0x5d7c18 0x49ec69 0x55e571 0x5d7cf1 0x49ced5 0x5d7c18 0x49ec69 0x55e571 0x55ef23 0x642140 0x6421be 0x644688 0x644c2c 0x6776be 0x677889 0x7fc2cce30c87 0x5e0f1a
Detected language: English
100% 24376/24376 [00:33<00:00, 728.62frames/s]


PARA **GRABAR** 

In [ ]:
Tarea = "Transcript to Language" #@param ["Transcript to Language", "Translate to English"]
import numpy as np
import whisper
from scipy.io.wavfile import write
from IPython.display import clear_output

task = "translate" if Tarea == "Translate to English" else "transcribe" 

audio, sr = get_audio()
write('record.wav', sr, audio)

!whisper "record.wav" --task {task} --model medium --verbose False

clear_output()
if task == "translate":
  print("-- TRADUCCIÓN A INGLÉS --\n")
else:
  print("-- TRANSCRIPCIÓN A ESPAÑOL --\n")
  
print(open('record.wav.txt').read())